このnotebookは以下のスコア0.699のnotebookに日本語訳を追加したものです。
日本人Kagglerの手助けになれば幸いです。
コメントか投票をいただけると嬉しいです。

This notebook is a Japanese translation of the following notebook with a score of 0.699. I hope it helps Japanese Kaggler. I would appreciate your comments or votes.

https://www.kaggle.com/code/vadimkamaev/catboost

In [1]:
import pandas as pd
import numpy as np
import pickle
import sys

from catboost import CatBoostClassifier

# Load Train Data and Labels

In [2]:
dtypes = {"session_id": 'int64',
          "index": np.int16,
          "elapsed_time": np.int32,
          "event_name": 'category',
          "name": 'category',
          "level": np.int8,
          "page": np.float16,
          "room_coor_x": np.float16,
          "room_coor_y": np.float16,
          "screen_coor_x": np.float16,
          "screen_coor_y": np.float16,
          "hover_duration": np.float32,
          "text": 'category',
          "fqid": 'category',
          "room_fqid": 'category',
          "text_fqid": 'category',
          "fullscreen": np.int8,
          "hq": np.int8,
          "music": np.int8,
          "level_group": 'category'
          }
use_col = ['session_id', 'index', 'elapsed_time', 'event_name', 'name', 'level', 'page',
           'room_coor_x', 'room_coor_y', 'hover_duration', 'text', 'fqid', 'room_fqid', 'text_fqid', 'level_group']

In [3]:
targets = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train_labels.csv')
targets['session'] = targets.session_id.apply(lambda x: int(x.split('_')[0]) )
targets['q'] = targets.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
# print( targets.shape )
# targets.head()

In [4]:
feature_df = pd.read_csv('/kaggle/input/featur/feature_sort.csv')

# Feature Engineer

df を 'session_id' 列と 'elapsed_time' 列でソートしています。inplace=True は、ソート結果を元の DataFrame に反映することを意味します。

'elapsed_time' 列の前の行との差分を計算して、'd_time' 列に代入しています。diff(1) は、前の行との差分を計算するメソッドです。

'd_time' 列の欠損値（NaN）を 0 で埋めています。fillna(0) は、欠損値を指定した値で埋めるメソッドです。

'd_time' 列の値を 0 から 103000 の範囲にクリップして、'delt_time' 列に代入しています。clip(0, 103000) は、値を指定した範囲内にクリップするメソッドです。

'delt_time' 列の値を 1 行後ろにシフトして、'delt_time_next' 列に代入しています。shift(-1) は、値を指定した数だけ行方向にシフトするメソッドです。

変更した DataFrame df を関数の結果として返します。

In [5]:
def delt_time_def(df):
    df.sort_values(by=['session_id', 'elapsed_time'], inplace=True)
    df['d_time'] = df['elapsed_time'].diff(1)
    df['d_time'].fillna(0, inplace=True)
    df['delt_time'] = df['d_time'].clip(0)
    df['delt_time'] = df['d_time'].clip(0, 103000)
    df['delt_time_next'] = df['delt_time'].shift(-1)
    return df

kol_col と kol_col_max というグローバル変数を定義しています。それぞれの値は、kol_col は 9 に、kol_col_max は 11+kol_f*2 に設定されます。kol_f は引数として渡される値です。

新しいデータフレーム new_train を作成しています。カラムの数は kol_col_max で指定された値になります。index=train['session_id'].unique() は、元のデータフレーム train の 'session_id' 列のユニークな値をインデックスとして使用します。

new_train の 10 列に 'session_id' を設定しています。

train データフレームを 'session_id' でグループ化し、'd_time' 列の値のパーセンタイルを計算して、new_train の 0～3 列に代入しています。

train データフレームを 'session_id' でグループ化し、'd_time' 列の値の平均値と標準偏差を計算して、new_train の 4～5 列に代入しています。

new_train の 10 列に 'session_id'が格納されているので、year, month, day, hourに分割して、new_train の 6～9 列に代入しています。

new_train の 10 列に0を代入しています。

new_train の 欠損値を-1で埋めます。

In [6]:
def feature_engineer(train, kol_f):
    global kol_col, kol_col_max
    kol_col = 9
    kol_col_max = 11+kol_f*2
    print('---'*10,kol_col_max)
    col = [i for i in range(0,kol_col_max)]
    new_train = pd.DataFrame(index=train['session_id'].unique(), columns=col, dtype=np.float16)  
    new_train[10] = new_train.index # "session_id"    
    print('---'*10,'newtrain',new_train[10])
    print('---'*10,'train',train)
    new_train[0] = train.groupby(['session_id'])['d_time'].quantile(q=0.3)
    new_train[1] = train.groupby(['session_id'])['d_time'].quantile(q=0.8)
    new_train[2] = train.groupby(['session_id'])['d_time'].quantile(q=0.5)
    new_train[3] = train.groupby(['session_id'])['d_time'].quantile(q=0.65)
    new_train[4] = train.groupby(['session_id'])['hover_duration'].agg('mean')
    new_train[5] = train.groupby(['session_id'])['hover_duration'].agg('std')    
    new_train[6] = new_train[10].apply(lambda x: int(str(x)[:2])).astype(np.uint8) # "year"
    new_train[7] = new_train[10].apply(lambda x: int(str(x)[2:4])+1).astype(np.uint8) # "month"
    new_train[8] = new_train[10].apply(lambda x: int(str(x)[4:6])).astype(np.uint8) # "day"
    new_train[9] = new_train[10].apply(lambda x: int(str(x)[6:8])).astype(np.uint8) + new_train[10].apply(lambda x: int(str(x)[8:10])).astype(np.uint8)/60
    new_train[10] = 0
    new_train = new_train.fillna(-1)
    
    return new_train

グローバル変数 kol_col の値を1増やしています。

row_f から 'col1' と 'val1' の値を取得し、条件式 train[col1] == val1 を用いて maska を作成しています。

row_f の 'kol_col' の値に応じて条件分岐を行っています。'kol_col' の値が 1 の場合は、train データフレームを maska でフィルタリングし、'delt_time_next' 列を 'session_id' でグループ化して合計値を計算し、それを new_train の kol_col 列に代入しています。さらに、gran_1 が真の場合は、同様に平均値を計算し、gran_2 が真の場合は、'index' 列の個数を計算して代入しています。

'kol_col' の値が 2 の場合も同様に処理が行われますが、追加で 'col2' と 'val2' の値を取得し、maska を更新しています。

最後に、new_train を返して関数の処理を終了します。

In [7]:
def feature_next_t(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    col1 = row_f['col1']
    val1 = row_f['val1']
    maska = (train[col1] == val1)
    if row_f['kol_col'] == 1:       
        new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = maska & (train[col2] == val2)        
        new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['index'].count()
    return new_train

グローバル変数 kol_col の値を1増やしています。

この行では、row_f から 'col1' と 'val1' の値を取得し、条件式 train[col1] == val1 を用いて maska を作成しています。

row_f の 'kol_col' の値に応じて条件分岐を行っています。'kol_col' の値が 1 の場合は、train データフレームを maska でフィルタリングし、'delt_time_next' 列の合計値を計算し、それを new_train の kol_col 列に代入しています。さらに、gran_1 が真の場合は、同様に平均値を計算し、gran_2 が真の場合は、'index' 列の個数を計算して代入しています。

'kol_col' の値が 2 の場合も同様に処理が行われますが、追加で 'col2' と 'val2' の値を取得し、maska を更新しています。

最後に、new_train を返して関数の処理を終了します。

In [8]:
def feature_next_t_otvet(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    col1 = row_f['col1']
    val1 = row_f['val1']
    maska = (train[col1] == val1)
    if row_f['kol_col'] == 1:      
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = maska & (train[col2] == val2)        
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()
    return new_train

グローバル変数 kol_col の値を1増やしています。

row_f の 'kol_col' の値に応じて条件分岐を行っています。'kol_col' の値が 1 の場合は、row_f の 'col1' 列と 'val1' の値に基づいて maska を作成しています。その後、train データフレームを maska でフィルタリングし、'delt_time_next' 列の合計値を計算して new_train の kol_col 列に代入しています。さらに、gran_1 が真の場合は、同様に平均値を計算し、gran_2 が真の場合は、'index' 列の個数を計算して代入しています。

'kol_col' の値が 2 の場合も同様に処理が行われますが、追加で 'col2' と 'val2' の値を取得し、maska を更新しています。

最後に、new_train を返して関数の処理を終了します。

In [9]:
def experiment_feature_next_t_otvet(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    if row_f['kol_col'] == 1: 
        maska = train[row_f['col1']] == row_f['val1']
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = (train[col1] == val1) & (train[col2] == val2)        
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()
    return new_train

グローバル変数 kol_col の値を 9 に設定し、g1 と g2 にそれぞれ 0.7 と 0.3 の値を割り当てています。

feature_df データフレームから 'quest' 列が quest と一致する行をフィルタリングして feature_q にコピーし、インデックスをリセットしています。

kol_f を g1 と g2 に乗じて四捨五入した値を gran1 と gran2 に設定しています。そして、0から kol_f-1 の範囲でループを回しています。各ループでは、feature_q の i 番目の行を row_f に代入し、feature_next_t_otvet 関数を呼び出しています。この関数には、row_f と各種パラメータを渡しており、それに基づいて new_train を更新しています。

最後に、new_train のカラム番号が 0 から kol_col までの範囲の部分を抽出して返しています。

このコードは、与えられた quest に対して特徴量の計算と追加を行う関数です。関数内で、feature_next_t_otvet 関数が呼び出されており、各行の特徴量の追加処理が行われています。また、関数の最後では特徴量が追加された new_train の一部を抽出して返しています。

In [10]:
def feature_quest_otvet(new_train, train, quest, kol_f):
    global kol_col
    kol_col = 9
    g1 = 0.7 
    g2 = 0.3 

    feature_q = feature_df[feature_df['quest'] == quest].copy()
    feature_q.reset_index(drop=True, inplace=True)
    
    gran1 = round(kol_f * g1)
    gran2 = round(kol_f * g2)    
    for i in range(0, kol_f):         
        row_f = feature_q.loc[i]
        new_train = feature_next_t_otvet(row_f, new_train, train, i < gran1, i <  gran2, i) 
    col = [i for i in range(0,kol_col+1)]
    return new_train[col]

g1 に 0.7、g2 に 0.3 の値を割り当て、kol_f をそれぞれ乗じて四捨五入した結果を gran1 と gran2 に代入しています。

0から kol_f-1 の範囲でループを回しています。各ループでは、feature_q の i 番目の行を row_f に代入し、feature_next_t 関数を呼び出しています。この関数には、row_f と各種パラメータを渡しており、それに基づいて new_train を更新しています。

最後に、更新された new_train を返しています。

このコードは、与えられた new_train と train データフレームに対して特徴量エンジニアリングを行う関数です。feature_q データフレームから行を取り出し、feature_next_t 関数を呼び出して特徴量の追加を行います。最終的に、更新された new_train を返します。

In [11]:
def feature_engineer_new(new_train, train, feature_q, kol_f):
    g1 = 0.7 
    g2 = 0.3 
    gran1 = round(kol_f * g1)
    gran2 = round(kol_f * g2)    
    for i in range(0, kol_f): 
        row_f = feature_q.loc[i]       
        new_train = feature_next_t(row_f, new_train, train, i < gran1, i <  gran2, i)         
    return new_train

グローバル変数 kol_col に値 9 を代入しています。

feature_df データフレームから 'quest' 列の値が quest と等しい行のみを抽出して、feature_q にコピーします。そして、インデックスをリセットしています。

feature_engineer_new 関数を呼び出して特徴量エンジニアリングを行い、その結果を new_train に代入しています。new_train と train データフレーム、feature_q、および kol_f を引数として渡しています。

インデックスが 0 から kol_col までの範囲を持つリストを作成し、new_train のそれらの列だけを抽出して返します。

このコードは、与えられた new_train と train データフレームに対して、指定された 'quest' 値に対応する特徴量エンジニアリングを行う関数です。feature_q データフレームから必要な行を抽出し、feature_engineer_new 関数を呼び出して特徴量の追加を行います。最後に、特定の列だけを持つ new_train を返します。

In [12]:
def feature_quest(new_train, train, quest, kol_f):
    global kol_col
    kol_col = 9
    feature_q = feature_df[feature_df['quest'] == quest].copy()
    feature_q.reset_index(drop=True, inplace=True)
    new_train = feature_engineer_new(new_train, train, feature_q, kol_f)
    col = [i for i in range(0,kol_col+1)]
    return new_train[col]

questsリストの要素数を kol_quest に代入しています。

questsリストをイテレーションしながら、各要素 q について処理を行います。また、q を出力しています。

feature_engineer 関数を使用して、old_train データフレームを特徴量エンジニアリングし、その結果を new_train に代入しています。list_kol_f[q] を feature_engineer の引数として渡しています。

feature_quest 関数を使用して、new_train と old_train データフレームを特徴量エンジニアリングし、その結果を train_x に代入しています。q と list_kol_f[q] を feature_quest の引数として渡しています。

train_x の形状を出力しています。

train_x のインデックス値を train_users に代入し、targets データフレームから q に対応する行を抽出して train_y に代入しています。

CatBoostClassifier モデルをインスタンス化して model に代入しています。

train_x を float32 型にキャストし、train_y['correct'] を正解ラベルとしてモデルを学習させます。

models ディクショナリに q をキーとして学習したモデルを追加します。

処理が終了したことを示す出力を行い、models を返します。

このコードは、与えられたデータと設定に基づいて複数のモデルを作成する関数です。各質問ごとに特徴量エンジニアリングを行い、トレーニングデータを作成し、CatBoostClassifier モデルを学習させます。学習済みモデルは models ディクショナリに格納され、最終的に返されます。

In [13]:
def create_model(old_train, quests, models, list_kol_f):
    
    kol_quest = len(quests)
    # ITERATE THRU QUESTIONS
    for q in quests:
        print('### quest ', q, end='')
        new_train = feature_engineer(old_train, list_kol_f[q])
        train_x = feature_quest(new_train, old_train, q, list_kol_f[q])
        print (' ---- ', 'train_q.shape = ', train_x.shape)
           
        # TRAIN DATA
        train_users = train_x.index.values
        train_y = targets.loc[targets.q==q].set_index('session').loc[train_users]

        # TRAIN MODEL 

        model = CatBoostClassifier(
            n_estimators = 300, #300
            learning_rate= 0.05,#0.045
            depth = 9
        )
        
        model.fit(train_x.astype('float32'), train_y['correct'], verbose=False)

        # SAVE MODEL, PREDICT VALID OOF
        models[f'{q}'] = model
    print('***')
    
    return models

In [14]:
models = {}
best_threshold = 0.63

In [15]:
list_kol_f = {
    1:140,3:110,
    4:110, 5:220, 6:120, 7:110, 8:110, 9:100, 10:120, 11:120,
    14: 110, 15:160, 16:105, 17:140             
             }

'/kaggle/input/featur/train_0_4t.csv' ファイルからデータを読み込み、df0_4 にデータフレームとして格納しています。dtypes を使用してデータ型を指定しています。

df0_4 データフレームを 'session_id' でグループ化し、各グループの 'level' のユニークな値の数が5未満かどうかを判定し、結果を kol_lvl に代入しています。

kol_lvl の値が True であるインデックスを抽出し、list_session に格納しています。

df0_4 データフレームから list_session に含まれる 'session_id' を持つ行を削除しています。

delt_time_def 関数を使用して df0_4 データフレームに対して時間の差分を計算し、新しい列を追加しています。

quests_0_4 リストに [1, 3] を代入しています。

create_model 関数を使用して df0_4 データフレームに対してモデルを作成し、結果を models に代入しています。

df0_4 を削除してメモリを解放しています。

このコードは、データの読み込みから特徴量の前処理、モデルの作成までの一連の処理を示しています。

In [16]:
df0_4 = pd.read_csv('/kaggle/input/featur/train_0_4t.csv', dtype=dtypes) 
kol_lvl = (df0_4 .groupby(['session_id'])['level'].agg('nunique') < 5)
list_session = kol_lvl[kol_lvl].index
df0_4  = df0_4 [~df0_4 ['session_id'].isin(list_session)]
df0_4 = delt_time_def(df0_4)

quests_0_4 = [1, 3] 
# list_kol_f = {1:140,3:110}

models = create_model(df0_4, quests_0_4, models, list_kol_f)
del df0_4

### quest  1------------------------------ 291
------------------------------ newtrain 20090312431273200    20090312431273200
20090312433251036    20090312433251036
20090312455206810    20090312455206810
20090313091715820    20090313091715820
20090313571836404    20090313571836404
                           ...        
22100215342220508    22100215342220508
22100215460321130    22100215460321130
22100217104993650    22100217104993650
22100219442786200    22100219442786200
22100221145014656    22100221145014656
Name: 10, Length: 23562, dtype: int64
------------------------------ train                 session_id  index  elapsed_time      event_name       name  \
0        20090312431273200      0             0  cutscene_click      basic   
2        20090312431273200      2           831    person_click      basic   
3        20090312431273200      3          1147    person_click      basic   
1        20090312431273200      1          1323    person_click      basic   
4        2009031243

In [17]:
df5_12 = pd.read_csv('/kaggle/input/featur/train_5_12t.csv', dtype=dtypes)
kol_lvl = (df5_12.groupby(['session_id'])['level'].agg('nunique') < 8)
list_session = kol_lvl[kol_lvl].index
df5_12 = df5_12[~df5_12['session_id'].isin(list_session)]
df5_12 = delt_time_def(df5_12)
quests_5_12 = [4, 5, 6, 7, 8, 9, 10, 11] 

# list_kol_f = {4:110, 5:220, 6:120, 7:110, 8:110, 9:100, 10:120, 11:120}

models = create_model(df5_12, quests_5_12, models, list_kol_f)
del df5_12

### quest  4------------------------------ 231
------------------------------ newtrain 20090312431273200    20090312431273200
20090312433251036    20090312433251036
20090312455206810    20090312455206810
20090313091715820    20090313091715820
20090313571836404    20090313571836404
                           ...        
22100215342220508    22100215342220508
22100215460321130    22100215460321130
22100217104993650    22100217104993650
22100219442786200    22100219442786200
22100221145014656    22100221145014656
Name: 10, Length: 23561, dtype: int64
------------------------------ train                 session_id  index  elapsed_time      event_name       name  \
0        20090312431273200    175        221485  navigate_click  undefined   
1        20090312431273200    176        222334    person_click      basic   
2        20090312431273200    177        223251    person_click      basic   
3        20090312431273200    178        223735  navigate_click  undefined   
4        2009031243

In [18]:
df13_22 = pd.read_csv('/kaggle/input/featur/train_13_22t.csv', dtype=dtypes) 
kol_lvl = (df13_22 .groupby(['session_id'])['level'].agg('nunique') < 10)
list_session = kol_lvl[kol_lvl].index
df13_22  = df13_22 [~df13_22 ['session_id'].isin(list_session)]
df13_22 = delt_time_def(df13_22)

quests_13_22 = [14, 15, 16, 17] 
# list_kol_f = {14: 110, 15:160, 16:105, 17:140}

models = create_model(df13_22, quests_13_22, models, list_kol_f)
del df13_22   #本notebookで追加

### quest  14------------------------------ 231
------------------------------ newtrain 20090312431273200    20090312431273200
20090312433251036    20090312433251036
20090312455206810    20090312455206810
20090313091715820    20090313091715820
20090313571836404    20090313571836404
                           ...        
22100215342220508    22100215342220508
22100215460321130    22100215460321130
22100217104993650    22100217104993650
22100219442786200    22100219442786200
22100221145014656    22100221145014656
Name: 10, Length: 22986, dtype: int64
------------------------------ train                  session_id  index  elapsed_time      event_name       name  \
0         20090312431273200    512        836732  navigate_click  undefined   
1         20090312431273200    513        837245  navigate_click  undefined   
2         20090312431273200    514        837779  navigate_click  undefined   
3         20090312431273200    515        838446  navigate_click  undefined   
4         200

**Infer Test Data**

特定の変数や属性をリセットするための処理が行われています。jo_wilder.make_env.__called__、env.__called__、type(env)._state の値がリセットされます。例外が発生した場合は、処理をスキップします。

jo_wilder.make_env() を呼び出して env を作成します。make_env は jo_wilder モジュール内の関数で、環境を初期化するための処理が含まれています。

env の iter_test() メソッドを呼び出して、イテレーターを取得しています。これにより、テストデータに対して反復処理を行うことができます。

このコードは、jo_wilder モジュールを使用して環境を初期化し、テストデータに対して反復処理を行うための準備を行っています。

In [19]:
import jo_wilder

try:
    jo_wilder.make_env.__called__ = False
    env.__called__ = False
    type(env)._state = type(type(env)._state).__dict__['INIT']
except:
    pass

env = jo_wilder.make_env()
iter_test = env.iter_test()    

In [20]:
import time

変数 g_end4 と g_end5 を初期化しています。

キーと値のペアを持つ辞書 list_q を作成しています。キーは文字列であり、値は quests_0_4、quests_5_12、quests_13_22 のいずれかのリストです。

iter_test を使ってテストデータに対して反復処理を行います。iter_test はテストデータのイテレーターであり、各反復では (test, sam_sub) のタプルが取得されます。

sam_sub の 'session_id' 列の値を処理して 'question' 列を作成します。'session_id' の値から数字を抽出し、'question' 列に格納します。

test データフレームの 'level_group' 列の最初の値を grp に代入します。

sam_sub データフレームの 'correct' 列を 1 で初期化し、特定の質問番号（5、8、10、13、15）に対応する行の 'correct' 列の値を 0 に設定します。

test データフレームから 'level_group' 列が grp と等しい行を選択し、その部分データフレームを delt_time_def 関数に渡して old_train を作成します。

grp に対応するキーを使って list_q 辞書から対応するリストを取得し、q として反復処理します。

特徴エンジニアリングの関数を使用して old_train から new_train を作成します。feature_engineer と feature_quest_otvet の呼び出しによって特徴量が追加され、計算時間 end4 が測定されます。

モデルを使用して new_train の予測を行います。モデルは models 辞書から取得され、predict_proba メソッドを使って確率の予測結果を取得します。計算時間 end5 も測定され、g_end5 に加算されます。

予測結果をもとに 'correct' 列の値を更新します。sam_sub.question == q の条件に一致する行の 'correct' 列の値を、p[0] と best_threshold の比較結果に基づいて更新します。

sam_sub データフレームから 'session_id' 列と 'correct' 列を選択し、それを使って env.predict() を呼び出します。これにより、予測結果が提出用に環境に送信されます。

このコードは、テストデータに対して予測を行い、その結果を提出用に環境に送信するための処理を実行しています。計算時間も測定され、g_end4 と g_end5 に加算されます。

In [21]:
g_end4 = 0
g_end5 = 0

list_q = {'0-4':quests_0_4, '5-12':quests_5_12, '13-22':quests_13_22}
for (test, sam_sub) in iter_test:
    sam_sub['question'] = [int(label.split('_')[1][1:]) for label in sam_sub['session_id']]    
    grp = test.level_group.values[0]   
    sam_sub['correct'] = 1
    sam_sub.loc[sam_sub.question.isin([5, 8, 10, 13, 15]), 'correct'] = 0  
    old_train = delt_time_def(test[test.level_group == grp])
       
    for q in list_q[grp]:
        
        start4 = time.time()
        new_train = feature_engineer(old_train, list_kol_f[q])
        new_train = feature_quest_otvet(new_train, old_train, q, list_kol_f[q])
#         new_train = feature_quest(new_train, old_train, q, kol_f)
        
        end4 = time.time() - start4
        g_end4 += end4
        
        start5 = time.time()        
        
        clf = models[f'{q}']
        p = clf.predict_proba(new_train.astype('float32'))[:,1]        
        
        end5 = time.time() - start5
        g_end5 += end5
             
        
        mask = sam_sub.question == q 
        x = int(p[0]>best_threshold)
        sam_sub.loc[mask,'correct'] = x      
        
        
    sam_sub = sam_sub[['session_id', 'correct']]      
    env.predict(sam_sub)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
------------------------------ 291
------------------------------ newtrain 20090109393214576    20090109393214576
Name: 10, dtype: int64
------------------------------ train             session_id  index  elapsed_time      event_name       name  level  \
0    20090109393214576      0             0  cutscene_click      basic      0   
1    20090109393214576      1          1965    person_click      basic      0   
2    20090109393214576      2          3614    person_click      basic      0   
3    20090109393214576      3          5330    person_click      basic      0   
4    20090109393214576      4          6397    person_click      basic      0   
..                 ...    ...           ...             ...        ...    ...   
135  20090109393214576    135        206861       map_click      basic      4   
136  20090109393214576    136        207212       map

# EDA submission.csv

In [22]:
# df = pd.read_csv('submission.csv')
# print( df.shape )
# df.head(60)